In [1]:
from __future__ import print_function, division, absolute_import

from copy import copy
# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model
import cobra_utils as wnql

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

from cobrame.core.component import (Constraint, ProcessedProtein, Complex,
                                    TranslatedGene, TranscribedGene, Metabolite, Ribosome,
                                    GenericComponent,GenerictRNA,RNAP)

print(cobra.__file__)
print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/Downloads/cobrapy-0.5.11/cobra/__init__.py
/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [8]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
bsub_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')
bsub_ref_directory = join(flat_files.ecoli_files_dir, 'Bacillus_subtilis_subsp_subtilis_str_168.xml')

eco = wnql.io.load_model(eco_directory,format='json')
bsub = cobra.io.load_json_model(bsub_directory)
bsub_ref = cobra.io.read_sbml_model(bsub_ref_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

Loading genome-scale model
Model correctly loaded.


<Solution 0.12 at 0x7f5bbd0af5f8>

In [3]:
with open('./me_models/iBS.pickle', 'rb') as f:
    me = pickle.load(f)
    
with open('./me_models/iJL1678b.pickle', 'rb') as f:
    eco_me = pickle.load(f)

In [4]:
GE_dict_m = gene_essentiality(bsub, model_type = 'm',  lim = 0.01, NP = 20)

Number of processors:  20
BSU07840 0 -2.9606060061953983e-06
BSU16730 0 -1.6845470178651136e-10
BSU16740 0 -1.6845470178651136e-10
BSU30820 e -1.0
BSU06310 0 -3.015210791159218e-10
BSU03040 0 -2.3118499904190636e-10
BSU38560 0 0.0
BSU40110 0 0.0
BSU03900 0 6.264465396683538e-11
BSU39860 0 0.0
BSU38830 0 0.0
BSU19310 0 0.0
BSU32830 0 -0.0009092498653858589
BSU24170 0 -0.00082955738714317
BSU22440 e -1.0000000000000207
BSU24350 e -1.0000000000000207
BSU29210 e -1.0000000000000207
BSU24340 e -1.0000000000000207
BSU29200 e -1.0000000000000207
BSU13900 - -0.01336478034198814
BSU13910 - -0.01336478034198814
BSU11200 e -1.0
BSU07700 0 -1.021307980261297e-10
BSU11210 e -1.0
BSU28300 0 0.0
BSU36010 0 0.0
BSU08090 0 0.0
BSU28310 0 0.0
BSU29690 0 0.0
BSU29710 0 0.0
BSU29470 0 -0.0015819159855061501
BSU29700 0 0.0
BSU36000 0 -9.532122761147827e-05
BSU08060 0 0.0
BSU18260 0 0.0
BSU32820 0 0.0
BSU08080 0 0.0
BSU08070 0 0.0
BSU30420 0 -1.5418838865418317e-10
BSU30440 0 -1.5418838865418317e-10
BSU3717

BSU27060 0 0.0
BSU13110 0 -4.1960159191678514e-05
BSU33040 - -0.13552338762256164
BSU04470 0 0.0
BSU03930 0 -3.8807115128571715e-11
BSU00500 e -1.0
BSU34550 0 -1.2365526222200466e-10
BSU28120 0 0.0
BSU08710 0 0.0
BSU35740 e -1.0
BSU22830 e -1.0
BSU09300 0 0.0
BSU13130 0 -5.235331878571987e-10
BSU16670 0 -4.321394465667102e-10
BSU35020 0 0.0
BSU02360 0 0.0
BSU23850 0 -1.9037585440137395e-10
BSU31350 - -0.01313449509840794
BSU02510 0 -3.892972149579262e-10
BSU02480 0 -4.895911526714601e-10
BSU38200 0 -7.754470390612869e-11
BSU30300 0 -2.4178370892679737e-10
BSU38190 0 -1.1343124172456196e-10
BSU35670 e -1.0
BSU12360 0 -1.1250210615719413e-10
BSU02350 0 -1.4916625680169745e-10
BSU33940 - -0.44636329977189554
BSU29020 0 -5.56485854833066e-10
BSU02230 0 6.720527672451433e-12
BSU30980 0 -8.177898808899397e-11
BSU01780 e -1.0
BSU36900 e -0.9999999986152125
BSU40070 0 0.0
BSU06510 0 -0.0006752543731877106
BSU30940 0 -2.672589403750037e-10
BSU02490 0 -7.575937073692459e-11
BSU30950 0 -1.2028846

BSU04740 e -1.0
BSU13730 0 -4.580400710525972e-10
BSU15020 0 -2.655268989773896e-10
BSU30530 0 0.0
BSU19630 0 0.0
BSU23490 0 0.0
BSU03070 0 0.0
BSU29180 - -0.036747515354280044
BSU00110 0 -1.441038914262895e-10
BSU00120 0 -1.441038914262895e-10
BSU39980 0 0.0
BSU27850 e -0.9999999761153312
BSU29300 0 0.0
BSU23270 0 -3.362171987517525e-10
BSU23250 0 -3.362171987517525e-10
BSU28790 0 -2.2956730555154488e-10
BSU28780 0 -2.70052935022381e-10
BSU31180 0 -1.6744098170750294e-10
BSU31200 0 6.17353998771689e-11
BSU17390 e -1.0
BSU17370 e -1.0
BSU17380 e -1.0
BSU36920 - -0.08802080373797043
BSU15590 e -1.0
BSU06160 0 -4.831917873311073e-10
BSU00930 0 -3.997064037743151e-10
BSU23770 0 -7.036172600383136e-11
BSU15850 0 -4.1793007210424855e-10
BSU15860 0 -4.1793007210424855e-10
BSU15630 0 -1.538226400496065e-10
BSU15620 0 7.758129053077435e-12
BSU25660 e -1.0
BSU03150 e -0.999999999999961
BSU11870 e -1.0
BSU15580 0 0.0
BSU01580 0 0.0
BSU15790 0 -0.0011324699729760221
BSU34660 0 0.0
BSU25020 0 -3.3

In [5]:
GE_dict_me = gene_essentiality(me, model_type = 'me',  lim = 0.01, NP = 20, initial_f=0)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 96.237275 seconds
Finished substituting S,lb,ub in 3.428821 seconds
Finished makeME_LP in 0.698816 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.5 0.5 1
Finished substituting S,lb,ub in 3.695689 seconds
Finished makeME_LP in 0.799448 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 3.540666 seconds
Finished makeME_LP in 0.687764 seconds
Getting MINOS parameters from ME_NLP...
3 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 3.366766 seconds
Finished makeME_LP in 0.670897 seconds
Getting MINOS parameters from ME_NLP...
4 0.0625 0.0625 0.125 0.0625 optimal
Finished substituting S,lb,ub in 3.386763 seconds
Finished makeME_LP in 0.711594 seconds
Getting MINOS parameters from ME_NLP...
5 0.09375 0.09375 0.125 0.09375 optimal
Finished substituting S,lb,ub in 3.503842 seconds
Finished makeME_LP in 0.690991 seconds
Getting MINOS parameters 

BSU22420 0 0.0
BSU22440 0 0.0
BSU22480 0 0.0
BSU22370 e -1.0
BSU22540 - -0.35714285714285715
BSU22360 e -1.0
BSU22550 - -0.35714285714285715
BSU22620 0 0.0
BSU22560 - -0.35714285714285715
BSU22490 e -1.0
BSU22610 e -1.0
BSU22600 e -1.0
BSU22630 e -1.0
BSU22680 0 0.0
BSU22690 0 0.0
BSU22640 e -1.0
BSU22660 e -1.0
BSU22650 e -1.0
BSU22670 e -1.0
BSU22700 e -1.0
BSU22870 0 0.0
BSU22890 0 0.0
BSU22740 e -1.0
BSU22960 0 0.0
BSU22760 e -1.0
BSU22750 e -1.0
BSU22710 e -1.0
BSU22730 e -1.0
BSU22780 e -1.0
BSU22830 e -1.0
BSU23100 0 0.0
BSU23080 e -1.0
BSU23450 0 0.0
BSU23070 e -1.0
BSU23270 e -1.0
BSU23490 0 0.0
BSU23250 e -1.0
BSU23500 0 0.0
BSU23260 e -1.0
BSU23580 0 0.0
BSU23610 0 0.0
BSU23560 0 0.0
BSU23570 0 0.0
BSU23760 0 0.0
BSU23280 e -1.0
BSU23800 0 0.0
BSU23770 0 0.0
BSU23850 0 0.0
BSU23380 e -1.0
BSU23860 0 0.0
BSU23920 0 0.0
BSU23840 e -1.0
BSU23960 0 0.0
BSU23980 0 0.0
BSU23970 0 0.0
BSU24070 0 0.0
BSU24120 0 0.0
BSU24090 0 0.0
BSU24130 0 0.0
BSU24030 e -1.0
BSU24140 0 0.0
BSU2415

In [10]:
# Create one dict
GE_dict = {}
for key in GE_dict_me.keys():
    GE_dict[key] = {}
    try:
        GE_dict[key]['m'] = GE_dict_m[key]
    except:
        GE_dict[key]['m'] = None
    GE_dict[key]['me'] = GE_dict_me[key]
    
for key in GE_dict_m.keys():
    if key not in GE_dict.keys():
        GE_dict[key] = {}
        GE_dict[key]['m'] = GE_dict_m[key]
        try:
            GE_dict[key]['me'] = GE_dict_me[key]
        except:
            GE_dict[key]['me'] = None        
GE_dict

{'BSU00110': {'m': '0', 'me': '0'},
 'BSU00180': {'m': '0', 'me': '0'},
 'BSU00140': {'m': '0', 'me': '0'},
 'BSU00410': {'m': None, 'me': '0'},
 'BSU00270': {'m': '0', 'me': '0'},
 'BSU00120': {'m': '0', 'me': '0'},
 'BSU00730': {'m': '0', 'me': '0'},
 'BSU00150': {'m': '0', 'me': '0'},
 'BSU00090': {'m': '0', 'me': '0'},
 'BSU00680': {'m': '0', 'me': '0'},
 'BSU00760': {'m': 'e', 'me': 'e'},
 'BSU00500': {'m': 'e', 'me': 'e'},
 'BSU00460': {'m': 'e', 'me': 'e'},
 'BSU00380': {'m': '0', 'me': 'e'},
 'BSU00740': {'m': 'e', 'me': 'e'},
 'BSU00750': {'m': 'e', 'me': 'e'},
 'BSU00130': {'m': None, 'me': 'e'},
 'BSU00280': {'m': 'e', 'me': 'e'},
 'BSU00550': {'m': None, 'me': 'e'},
 'BSU00510': {'m': 'e', 'me': 'e'},
 'BSU00810': {'m': None, 'me': '0'},
 'BSU00980': {'m': None, 'me': '0'},
 'BSU00930': {'m': '0', 'me': '0'},
 'BSU00770': {'m': 'e', 'me': 'e'},
 'BSU00790': {'m': 'e', 'me': 'e'},
 'BSU00820': {'m': None, 'me': 'e'},
 'BSU00780': {'m': 'e', 'me': 'e'},
 'BSU01070': {'m': Non

In [11]:
with open("./me_models/GE_dict.pickle", "wb") as outfile:
    pickle.dump(GE_dict, outfile)

In [10]:
r = get_reactions_of_met(me,'protein_BSU04560',0)

( formation_BSU04560-MONOMER 0.0 1000.0 ) 	 protein_BSU04560 --> BSU04560-MONOMER
( translation_BSU04560 0.0 1000.0 ) 	 10fthf_c + 0.00150854700854701*mu BSU01120-MONOMER + 4.27350427350427e-6*mu BSU01390-MONOMER + 4.27350427350427e-6*mu BSU15720-MONOMER + 4.27350427350427e-6*mu BSU15730-MONOMER + 0.00150854700854701*mu BSU16500-MONOMER + 4.27350427350427e-6*mu BSU16520-MONOMER + 8.54700854700855e-6*mu BSU16630-MONOMER + 4.27350427350427e-6*mu BSU28870-MONOMER + 4.27350427350427e-6*mu BSU35290-MONOMER + 0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_BSU04560 + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 33 ala__L_c + 9 arg__L_c + 14 asn__L_c + 19 asp__L_c + 353.0 + 1.59089163237311*(mu + 0.3915)/mu atp_c + 4 cys__L_c + 4.27350427350427e-6*mu generic_RF + 0.00150854700854701*mu generic_Tuf + 20.0 generic_tRNA_AAA_lys__L_c + 9.0 generic_tRNA_AAC_asn__L_c + 5.0 generic_tRNA_AAG_lys__L_c + 5.0 generic_tRNA_AAU_asn__L_c + 8.0 gen

In [17]:
bsub.solution.f

0.11796638935761272

In [61]:
get_metabolites_from_pattern(me,'cl_c')

['shcl_c']

In [113]:
met = 'protein_b3035'
print(eco_me.metabolites.get_by_id(met).name)
print_reactions_of_met(eco_me,met,1)

In [6]:
met = 'fmn_c'
print(me.metabolites.get_by_id(met).name)
print_reactions_of_met(me,met,-1)

FMN C17H19N4O9P


NameError: name 'print_reactions_of_met' is not defined

In [57]:
for met in bsub.metabolites:
    if 'complex' in met.name:
        print(met.id, met.name)

In [15]:
eco_me.metabolites.get_by_id('GRXB-MONOMER_mod_Oxidized').formula_weight

24349.923540000003

In [5]:
eco_me.metabolites.get_by_id('heme_c').formula

KeyError: 'heme_c'

In [15]:
bsub.metabolites.get_by_id('ACP_c').formula

'HX'

In [42]:
me.reactions.DNA_replication.metabolites

{<Metabolite datp_c at 0x7f6228fa36a0>: -0.246051501452,
 <Metabolite dctp_c at 0x7f6228fa35f8>: -0.253948498548,
 <Metabolite dgtp_c at 0x7f6228fa31d0>: -0.253948498548,
 <Metabolite dttp_c at 0x7f6228fa33c8>: -0.246051501452,
 <Metabolite ppi_c at 0x7f622fe6b828>: 1,
 <Constraint DNA_biomass at 0x7f622d17b6a0>: 0.3079428373534753}

In [37]:
r = get_reactions_of_met(bsub,'ac_c',0)

( ACS 0.0 999999.0 ) 	 ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
( ADPDA -999999.0 999999.0 ) 	 26dapime_c + h2o_c <=> 26dap_LL_c + ac_c
( ALDD2x 0.0 999999.0 ) 	 acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
( ACODA_1 0.0 999999.0 ) 	 acorn_c + h2o_c --> ac_c + orn__L_c
( CYSS_2 0.0 999999.0 ) 	 acser_c + h2s_c --> ac_c + cys__L_c
( AHSERL4 -999999.0 999999.0 ) 	 acser_c + trdrd_c + tsul_c <=> ac_c + cys__L_c + h_c + so3_c + trdox_c
( AGDC 0.0 999999.0 ) 	 acgam6p_c + h2o_c --> ac_c + gam6p_c
( AHSERL3 -999999.0 999999.0 ) 	 acser_c + seln_c <=> ac_c + scys__L_c
( SLCYSS 0.0 999999.0 ) 	 acser_c + tsul_c --> ac_c + scys__L_c
( ACKr -999999.0 999999.0 ) 	 ac_c + atp_c <=> actp_c + adp_c
( ACt2r -999999.0 999999.0 ) 	 ac_e + h_e <=> ac_c + h_c


In [31]:
i=0
for rxn in me.reactions:
    rule = rxn.gene_reaction_rule
    if 'dummy' in rxn.id:
        i = i + 1

print(i)

182


In [7]:
eco_me.reactions.ATPM.gene_reaction_rule

''

In [25]:
rxn_id = 'PYRt2rpp'
eco.reactions.get_by_id(rxn_id).reaction
eco_me.reactions.SELGTHR2_FWD_CPLX_dummy.gene_reaction_rule

''